# Azure Open AI - Demo 4 Speech summarizing

**Before getting started!**
- Create an Azure Speech service and create GitHub Secrets for it (or use and .env file uncommenting some sections)
    - AZURE_SPEECH_KEY and AZURE_SPEECH_REGION
- Make sure speech module **azure-cognitiveservices-speech** is installed (already executed automatically in the GH Codespace from requirements.txt)

In [1]:
import azure.cognitiveservices.speech as speechsdk
import openai
import os
import sys
import time

# from dotenv import load_dotenv
from IPython.display import Audio

In [2]:
sys.version

'3.11.5 (main, Sep 26 2023, 13:12:46) [GCC 10.2.1 20210110]'

In [3]:
print("openai version =", openai.__version__)

openai version = 0.28.1


In [4]:
# load_dotenv("azure.env")

openai.api_type: str = "azure"
# openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
# openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
# openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_version = os.getenv("AZURE_OPENAI_MODEL_CHAT_VERSION")

openai.engine = os.getenv("AZURE_OPENAI_MODEL_CHAT")

azure_speech_key = os.getenv("AZURE_SPEECH_KEY")
azure_speech_region = os.getenv("AZURE_SPEECH_REGION")

## 1. Convert Audio To Text Using Azure Speech To Text  

Recognize feature provided by Azure Speech to convert Audio to Text

In [5]:
def recognize_speech_from_file(filename):
    """This function transcribes an audio file into its corresponding textual output."""

    # setup global parameters
    global done
    done = False
    global recognized_text_list
    recognized_text_list = []

    # initiate the Speech Service
    speech_config = speechsdk.SpeechConfig(
        subscription=azure_speech_key, region=azure_speech_region
    )
    audio_config = speechsdk.audio.AudioConfig(filename=filename)

    # create a Speech Recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config, audio_config=audio_config
    )

    # define a call-back stop function
    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print("CLOSING on {}".format(evt))
        global done
        done = True

    # define a call-back recognize function
    def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        """callback for recognizing the recognized text"""
        global recognized_text_list
        recognized_text_list.append(evt.result.text)
        print("RECOGNIZED: {}".format(evt.result.text))

    # connect callbacks to the recognizer events
    speech_recognizer.recognized.connect(recognize_cb)
    speech_recognizer.session_started.connect(
        lambda evt: print("STT SESSION STARTED: {}".format(evt))
    )
    speech_recognizer.session_stopped.connect(
        lambda evt: print("STT SESSION STOPPED {}".format(evt))
    )

    # stop continuous speech recognition
    speech_recognizer.session_stopped.connect(stop_cb)

    # start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(0.5)

    speech_recognizer.stop_continuous_recognition()

    return recognized_text_list

## 2. Summarize Audio - Positive Example 

In [6]:
# play the audio sound
Audio("documents/good_review.wav", autoplay=False)

In [7]:
text = recognize_speech_from_file("documents/good_review.wav")

STT SESSION STARTED: SessionEventArgs(session_id=40a2709837244347b19f22336ea111a7)
RECOGNIZED: I had purchased another thermostat from a big box store and the Rep there assured me that it would turn off when it got too cold.
RECOGNIZED: Well, come to find out, it did not and my heater in the garage was running a lot of the time.
RECOGNIZED: This one was great and was easily wired.
RECOGNIZED: It does turn off when you turn it all the way to the left.
RECOGNIZED: It's annoying they don't have any in Celsius, but whatever.
RECOGNIZED: It runs well and actually turns off my garage heater.
RECOGNIZED: Would totally purchase again.
RECOGNIZED: Installed with five kilowatts heater in garage.
RECOGNIZED: Thermostat failed an on position overnight and came close to fire with overheated connections. See photos.
RECOGNIZED: Installation was by certified electrician.
STT SESSION STOPPED SessionEventArgs(session_id=40a2709837244347b19f22336ea111a7)
CLOSING on SessionEventArgs(session_id=40a2709837

In our case we will use the GPT3 model used in other demos

In [15]:
#model = "text-davinci-003"
print("openai.engine =", openai.engine)

openai.engine = gpt-35-turbo-unai


Analyze the provided Audio

In [18]:
# create a prompt for text analytics
prompt = f"""Text:

             ###
             {" ".join(text)}
             ###
             
             Tasks:
             1. Summarize the above text in a short and concise manner.
             2. Determine the above text as whether Positive or Negative.
             3. Classify the above text into a topic.
             4. Whether the above text is Positive or Negative, 
                suggest 3 ways to handle it.
                
             Answer the above 4 tasks, one at a time. 
          """
# results = openai.Completion.create(
#     engine=model, prompt=prompt, temperature=0, max_tokens=1000
# )
results = openai.Completion.create(
     engine=openai.engine, prompt=prompt, temperature=0, max_tokens=1000
)

print(results["choices"][0]["text"])

 ''')

# Answer 1
st.write('1. Summarize the above text in a short and concise manner.')
st.write('The customer had purchased a thermostat from a big box store which did not turn off when it got too cold. The customer then purchased another thermostat which was easily wired and turned off when turned all the way to the left. The customer would totally purchase it again. However, the thermostat failed in an on position overnight and came close to fire with overheated connections. The installation was done by a certified electrician.')

# Answer 2
st.write('2. Determine the above text as whether Positive or Negative.')
st.write('The text is both Positive and Negative. Positive because the customer was happy with the second thermostat and would totally purchase it again. Negative because the thermostat failed in an on position overnight and came close to fire with overheated connections.')

# Answer 3
st.write('3. Classify the above text into a topic.')
st.write('The above text is related

## 3. Summarize Audio - Negative Example 

In [19]:
Audio("documents/bad_review.wav", autoplay=False)

In [20]:
text = recognize_speech_from_file("documents/bad_review.wav")

STT SESSION STARTED: SessionEventArgs(session_id=f56ec40696be4c78b0a4cb122ce6ed64)
RECOGNIZED: Installed with five kilowatts heater in garage.
RECOGNIZED: Thermostat failed in on position overnight and came close to fire with overheated connections. See photos.
RECOGNIZED: Installation was by certified electrician.
STT SESSION STOPPED SessionEventArgs(session_id=f56ec40696be4c78b0a4cb122ce6ed64)
CLOSING on SessionEventArgs(session_id=f56ec40696be4c78b0a4cb122ce6ed64)


In [22]:
prompt = f"""Text:

             ###
             {" ".join(text)}
             ###
             
             Tasks:
             1. Summarize the above text in a short and concise manner.
             2. Determine the above text as whether Positive or Negative.
             3. Classify the above text into a topic.
             4. Whether the above text is Positive or Negative, 
                suggest 3 ways to handle it.
                
             Answer the above 4 tasks, one at a time. 
          """

# openai results
results = openai.Completion.create(
    engine=openai.engine, prompt=prompt, temperature=0, max_tokens=1000
)

print(results["choices"][0]["text"])

 ''')

# Answer 1
print('Answer 1: Summarize the above text in a short and concise manner.')
print('Answer: The installed heater in the garage came close to fire due to the thermostat failure in the on position overnight. The installation was done by a certified electrician.')
print('\n')

# Answer 2
print('Answer 2: Determine the above text as whether Positive or Negative.')
print('Answer: Negative')
print('\n')

# Answer 3
print('Answer 3: Classify the above text into a topic.')
print('Answer: Home Improvement')
print('\n')

# Answer 4
print('Answer 4: Whether the above text is Positive or Negative, suggest 3 ways to handle it.')
print('Answer: ')
print('1. The thermostat should be checked regularly to avoid any malfunctioning.')
print('2. The heater should be installed in a safe place to avoid any fire hazard.')
print('3. The installation should be done by a certified electrician only.')
print('\n')

# End of the program
print('End of the program')<|im_sep|>
